<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-2/Insumos/M%C3%A9todo_de_Momentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import pandas as pd

#archivo = pd.read_excel("/content/Historico Deudores (modelo de valores de referencia).xls") 

In [0]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 557 (delta 6), reused 0 (delta 0), pack-reused 541
Receiving objects: 100% (557/557), 25.10 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [0]:
%%R
# install.packages("rootSolve")
# install.packages("bivariate")
library("MASS")
library(rootSolve)
library(bivariate)
D_t <- matrix(c(0,2,1,2,0,2,0,0,2,2,3,0,0,1,1,0,2,4,4,5,6,14,3,0,1))
N_t <- matrix(c(1070,1099,1122,1181,1216,1337,1334,1349,1400,1447,1496,1661,1822,1916,2144,2336,2551,2718,2790,2835,2929,3049,2998,3117,3264))
PMI <- colMeans(D_t/N_t) #Promedio de la fórmula para PMI
print("La Probabilidad Media de Incumplimiento está dada por:")
print(PMI)
Aux_PCI <- matrix(0, nrow = 25, ncol=1)
Aux_PCI <- (D_t*(D_t-1))/(N_t*(N_t-1))
PCI <- colMeans(Aux_PCI) #Promedio de la fórmula para PCI
print("La Probabilidad Conjunta de Incumplimiento está dada por:")
print(PCI)
VCR <- qnorm(PMI)
print("Nuestro valor crítico de referencia es:")
print(VCR)
#Suponiendo una correlación de activos de 0.2
print("Suponiendo que nuestra de activos tiene un valor de 0.2")
Corr_Act <- 0.2
Factor_sens <- sqrt(Corr_Act)
print("El factor de sensibilidad estaría dado por:")
print(Factor_sens)
PCI_normbiv = nbvcdf(0,0,1,1,Corr_Act)
print("Y la Probabilidad Conjunta de Incumplimiento de la Normal Bivariada Estándar:")
print(PCI_normbiv(VCR,VCR))
print(nbvcdf(0,0,1,1,Corr_Act)(VCR,VCR)) 


[1] "La Probabilidad Media de Incumplimiento está dada por:"
[1] 0.001004205
[1] "La Probabilidad Conjunta de Incumplimiento está dada por:"
[1] 1.543424e-06
[1] "Nuestro valor crítico de referencia es:"
[1] -3.088986
[1] "Suponiendo que nuestra de activos tiene un valor de 0.2"
[1] "El factor de sensibilidad estaría dado por:"
[1] 0.4472136
[1] "Y la Probabilidad Conjunta de Incumplimiento de la Normal Bivariada Estándar:"
[1] 6.938907e-06
[1] 6.938907e-06


Encontremos el valor de Corr_Act que hace lo más cercano a 0 nuestro PCI_normbiv

In [0]:
%%R
#f <- function(x)  (1/(2*pi*sqrt(1-(x)^2)))*exp(-(1/(2*(1-(x^2))))*((2*(VCR^2))-(2*Corr_Act*VCR*VCR)))-PCI
G <- function(x)  nbvcdf(0,0,1,1,x)(VCR,VCR)/PCI-1
#Resuelve el sistema
raiz = multiroot(G,0.2,positive = TRUE, maxiter=10000) 
print("Encontramos el valor aproximado a 0, el cual nos representa la Probabilidad Conjunta de Incumplimiento por Normal Bivariada y que nos resulta:")
print(G(raiz$root))
print("Donde el valor de la Correlación de Activos que encontramos está dada por: ")
print(raiz$root)
print("Es decir, nuestro factor de sensibilidad es: ")
print(sqrt(raiz$root))
print("Valor bivariado ")
print( nbvcdf(0,0,1,1,raiz$root)(VCR,VCR))

[1] "Encontramos el valor aproximado a 0, el cual nos representa la Probabilidad Conjunta de Incumplimiento por Normal Bivariada y que nos resulta:"
[1] 8.70265e-10
[1] "Donde el valor de la Correlación de Activos que encontramos está dada por: "
[1] 0.03886633
[1] "Es decir, nuestro factor de sensibilidad es: "
[1] 0.1971455
[1] "Valor bivariado "
[1] 1.543424e-06


Como podemos observar, la Probabilidad Media de Incumplimiento y la Probabilidad Conjunta de Incumplimiento resultaron iguales a las calculadas en el archivo de Excel. Sin embargo, a la hora de aproximar la correlación de activos, los resultados aquí son muy altos comparados a los obtenidos en la hoja de cálculo, desde la función que utilizamos para estimar la función de probabilidad acumulada de la normal bivariada, los resultados varían un poco en los decimales. Pero el factor de sensibilidad mostrado aquí arriba resultó más de tres veces mayor al obtenido en la hoja de cálculo.
En general, los resultados muestran que la probabilidad media y, sobre todo, una probabilidad conjunta de incumplimiento con valores pequeños, lo cual nos da cierta seguridad en cuanto a incumplimiento se refiere.
Respecto a la correlación de activos, ambas son positivas aunque, una de ellas es relativamente pequeña y nos muestra una correlación casi nula, el otro resultado nos da a entender que su fuerza de relación hacia la misma dirección es moderada.